### Import of all libraries

In [1]:
import pandas as pd
import re
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from itertools import combinations
import datetime
import squarify

### Read csv file and basic info of data

In [2]:
df = pd.read_csv('Motor_Vehicle_Collisions_Person.csv',dtype={'ZIP CODE':'str'})
pd.set_option('display.max_columns', None)

In [3]:
df.shape

(5388842, 21)

In [4]:
df.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5388842 entries, 0 to 5388841
Data columns (total 21 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   UNIQUE_ID              5388842 non-null  int64  
 1   COLLISION_ID           5388842 non-null  int64  
 2   CRASH_DATE             5388842 non-null  object 
 3   CRASH_TIME             5388842 non-null  object 
 4   PERSON_ID              5388823 non-null  object 
 5   PERSON_TYPE            5388842 non-null  object 
 6   PERSON_INJURY          5388842 non-null  object 
 7   VEHICLE_ID             5169782 non-null  float64
 8   PERSON_AGE             4809562 non-null  float64
 9   EJECTION               2765258 non-null  object 
 10  EMOTIONAL_STATUS       2848154 non-null  object 
 11  BODILY_INJURY          2848197 non-null  object 
 12  POSITION_IN_VEHICLE    2765645 non-null  object 
 13  SAFETY_EQUIPMENT       2765556 non-null  object 
 14  PED_LOCATION      

In [5]:
df['PED_ROLE'].value_counts()

Registrant         2234689
Driver             1977950
Passenger           782325
Pedestrian           86152
Witness              72959
Owner                26937
Notified Person       8463
Policy Holder         2415
Other                 1702
In-Line Skater         361
Name: PED_ROLE, dtype: int64

### Filling all NaN values

In [6]:
u = "Unspecified"

df['PERSON_AGE'] = df['PERSON_AGE'].fillna(-1).astype(int)
df['PERSON_AGE'] = df['PERSON_AGE'].clip(lower=-1, upper=100)#Values 0 and 100 are considered to be Unspecified

mapping = {
    'Unknown' : 'Unspecified'
}
df['EJECTION'] = df['EJECTION'].fillna(u).replace(mapping)

mapping = {
    'Unknown' : 'Unspecified',
    'Does Not Apply' : 'Unspecified',
    'If one person is seated on another person&apos;s lap' : 'Seated on anothers lap',
    '-' : 'None',
    'Lap Belt/Harness' : 'Lap Belt & Harness',
    'Air Bag Deployed/Lap Belt' : 'Air Bag Deployed',
    'Other' : 'Unspecified',
    'M' : 'Male',
    'F' : 'Female',
    'U' : 'Unspecified'
}

df['CRASH_DATE'] = pd.to_datetime(df['CRASH_DATE'])
df['EMOTIONAL_STATUS'] = df['EMOTIONAL_STATUS'].fillna(u).replace(mapping)
df['BODILY_INJURY'] = df['BODILY_INJURY'].fillna(u).replace(mapping)
df['POSITION_IN_VEHICLE'] = df['POSITION_IN_VEHICLE'].fillna(u).replace(mapping)
df['SAFETY_EQUIPMENT'] = df['SAFETY_EQUIPMENT'].fillna(u).replace(mapping)
df['PED_LOCATION'] = df['PED_LOCATION'].fillna(u).replace(mapping)
df['PED_ACTION'] = df['PED_ACTION'].fillna(u).replace(mapping)
df['PERSON_SEX'] = df['PERSON_SEX'].fillna(u).replace(mapping)
df['COMPLAINT'] = df['COMPLAINT'].fillna(u).replace(mapping)
df['CONTRIBUTING_FACTOR_1'] = df['CONTRIBUTING_FACTOR_1'].fillna(u).replace(mapping)
df['CONTRIBUTING_FACTOR_2'] = df['CONTRIBUTING_FACTOR_2'].fillna(u).replace(mapping)
df['PED_ROLE'] = df['PED_ROLE'].fillna(u).replace(mapping)

In [7]:
df['CRASH_DATE'] = pd.to_datetime(df['CRASH_DATE'])
df['day'] = df['CRASH_DATE'].dt.day
df['month'] = df['CRASH_DATE'].dt.month
df['year'] = df['CRASH_DATE'].dt.year

# df = df[(df['CRASH_DATE'] > '2012') & (df['CRASH_DATE'] < '2024')]
# df.sort_values(by='CRASH_DATE',inplace=True)
# df.reset_index(drop=True,inplace=True)


df = df[(df['year'] > 2012) & (df['year'] < 2024)]
df.sort_values(by='CRASH_DATE',inplace=True)
df.reset_index(drop=True,inplace=True)

In [8]:
df.head()

,UNIQUE_ID,COLLISION_ID,CRASH_DATE,CRASH_TIME,PERSON_ID,PERSON_TYPE,PERSON_INJURY,VEHICLE_ID,PERSON_AGE,EJECTION,EMOTIONAL_STATUS,BODILY_INJURY,POSITION_IN_VEHICLE,SAFETY_EQUIPMENT,PED_LOCATION,PED_ACTION,COMPLAINT,PED_ROLE,CONTRIBUTING_FACTOR_1,CONTRIBUTING_FACTOR_2,PERSON_SEX,day,month,year
0,1894053,199014,2013-01-01,2:45,1,Pedestrian,Injured,NaN,26,Unspecified,Unspecified,Unspecified,Unspecified,Unspecified,Unspecified,Unspecified,Unspecified,Unspecified,Unspecified,Unspecified,Unspecified,1,1,2013
1,1987609,3017187,2013-01-01,23:45,1,Occupant,Injured,12838250.0,65,Unspecified,Unspecified,Unspecified,Unspecified,Unspecified,Unspecified,Unspecified,Unspecified,Unspecified,Unspecified,Unspecified,Unspecified,1,1,2013
2,2089241,256335,2013-01-01,3:00,1,Occupant,Injured,13527705.0,0,Unspecified,Unspecified,Unspecified,Unspecified,Unspecified,Unspecified,Unspecified,Unspecified,Unspecified,Unspecified,Unspecified,Unspecified,1,1,2013
3,2104150,260764,2013-01-01,15:30,1,Occupant,Injured,13647953.0,45,Unspecified,Unspecified,Unspecified,Unspecified,Unspecified,Unspecified,Unspecified,Unspecified,Unspecified,Unspecified,Unspecified,Unspecified,1,1,2013
4,2049209,240702,2013-01-01,1:40,1,Occupant,Injured,13231141.0,39,Unspecified,Unspecified,Unspecified,Unspecified,Unspecified,Unspecified,Unspecified,Unspecified,Unspecified,Unspecified,Unspecified,Unspecified,1,1,2013


### Change names of columns to increase efficiency

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5223189 entries, 0 to 5223188
Data columns (total 24 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   UNIQUE_ID              int64         
 1   COLLISION_ID           int64         
 2   CRASH_DATE             datetime64[ns]
 3   CRASH_TIME             object        
 4   PERSON_ID              object        
 5   PERSON_TYPE            object        
 6   PERSON_INJURY          object        
 7   VEHICLE_ID             float64       
 8   PERSON_AGE             int32         
 9   EJECTION               object        
 10  EMOTIONAL_STATUS       object        
 11  BODILY_INJURY          object        
 12  POSITION_IN_VEHICLE    object        
 13  SAFETY_EQUIPMENT       object        
 14  PED_LOCATION           object        
 15  PED_ACTION             object        
 16  COMPLAINT              object        
 17  PED_ROLE               object        
 18  CONTRIBUTING_FACTOR_1 

In [10]:
df.rename(columns = {'UNIQUE_ID' : 'UniqueID',
                     'COLLISION_ID' : 'CollisionID',
                     'CRASH_DATE' : 'CrashDate',
                     'CRASH_TIME' : 'CrashTime',
                     'PERSON_ID' : 'PersonID',
                     'PERSON_TYPE' : 'PersonType',
                     'PERSON_INJURY' : 'PersonInjury',
                     'VEHICLE_ID' : 'VehicleID',
                     'PERSON_AGE' : 'PersonAge',
                     'EJECTION' : 'Ejection',
                     'EMOTIONAL_STATUS' : 'EmotionalStatus',
                     'BODILY_INJURY' : 'BodilyInjury',
                     'POSITION_IN_VEHICLE' : 'PositionInVehicle',
                     'SAFETY_EQUIPMENT' : 'SafetyEquipment',
                     'PED_LOCATION' : 'PedLocation',
                     'PED_ACTION' : 'PedAction',
                     'COMPLAINT' : 'Complaint',
                     'PED_ROLE' : 'PedRole',
                     'CONTRIBUTING_FACTOR_1' : 'ContributingFactor1',
                     'CONTRIBUTING_FACTOR_2' : 'ContributingFactor2',
                     'PERSON_SEX' : 'PersonSex'
          }, inplace = True)

## Save to file

In [11]:
#df.to_csv('Person_FINAL.csv', sep=',', index=False, encoding='utf-8')